In [27]:
import torch
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer

In [56]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import tensorflow_hub as hub

In [29]:
model = DistilBertModel.from_pretrained('cc_pretrained')

In [30]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [58]:
tf_hub_embed = hub.load("/Users/harinisuresh/Dev/tf_hub/universal-sentence-encoder_4/") # change path here

In [ ]:
old_embeddings = np.load('../react-app/api/data/reddit/funny/embs.npy')
old_embeddings_subset = old_embeddings[:10000]

In [ ]:
def get_top_comments(sentence, embed_model='use'):
    if embed_model == 'use':
        vec = tf_hub_embed(sentence)
        top_comments = getKNNFromVector(vec, old_embeddings_subset)
    elif embed_model == 'cc':
        vec = tokenizer(sentence, return_tensors="pt", max_length=300, truncation=True, padding="max_length")
        top_comments = getKNNFromVector(vec, all_embeddings)
    return top_comments

In [31]:
all_comments = pd.read_csv('../react-app/api/data/reddit/funny/projection_df').comment

In [59]:
subset_comments = all_comments[:10000]

In [60]:
tokenized_comments = tokenizer(
        list(subset_comments), return_tensors="pt", max_length=300, truncation=True, padding=True
    )

In [ ]:
data_length = len(list(subset_comments))
batch_size = 10
all_embeddings = np.zeros((data_length, 768))

for batch_idx in range(0, int(data_length/batch_size)):
    print(batch_idx)
    min_idx = batch_size*batch_idx
    max_idx = min(data_length, batch_size*(batch_idx+1))
    batch = {'input_ids': tokenized_comments.input_ids[min_idx:max_idx], 
             'attention_mask': tokenized_comments.input_ids[min_idx:max_idx]}
    
    all_embeddings[min_idx:max_idx] = model(**batch).last_hidden_state.detach().numpy().mean(axis=1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
np.save('cc_embs.npy', all_embeddings)

In [40]:
old_embeddings = np.load('../react-app/api/data/reddit/funny/embs.npy')

In [42]:
old_embeddings_subset = old_embeddings[:1000]

In [ ]:
test = "i hate men"

In [ ]:
test_emb_new = 

In [ ]:
def getKNNFromVector(vec, embeddings, n=30):
    dist_vec = cdist([vec], embeddings, 'cosine')
    top_vec_idx = np.argsort(dist_vec[0])[1:n+1]    
    top_comments = np.array(subset_comments)[top_vec_idx]
    return top_comments

In [50]:
print(subset_comments[0])
getKNNFromVector(old_embeddings_subset[0], old_embeddings_subset)

You can tell this is fake because it shows a black father in the picture
[ 13  27  35  26 325 511 711 759  23  31 151  21  10 305 452  30 787   5
 219 128  85 353   6 610 698 849 877 623 937 572]
["FAKE!!! Black babies don't have dad."
 "It's so rare to see a black man with its kid that it's always guaranteed to make the front page."
 'Fake.  No such thing an entire nucleus black family.  \n\nReported.'
 "It's a Christmas miracle. A black father who stuck around after the kid was born. Praise Jesus."
 'Did OP really just take one of the top all time posts on /r/bannedfromclubpenguin  and Photoshop his joke into it the picture? http://i.imgur.com/qgUia.jpg'
 'its funny how millennials get upset when you point out that the pictures they post on the internet are really fucking old. Even when you know the dude in the pic, and its from over a decade ago. \n\nlol'
 "He's not... that looks nothing like him you faggot"
 'Yet if someone impersonated Obama and posted it here, it would be racist.

In [51]:
getKNNFromVector(all_embeddings[0], all_embeddings)

[  6  35 339  26 275  25  31   5 479 633  30 759  27 274 771 128 914 963
  43  40  18 795 531 764 661 481 141 993 630 712]
['Why is your Santa white? I watch the news and thought Santa was black...'
 'Fake.  No such thing an entire nucleus black family.  \n\nReported.'
 "What's a word that starts with 'n' that you never want to call a black person.\n\nneighbor"
 "It's a Christmas miracle. A black father who stuck around after the kid was born. Praise Jesus."
 '> So you equate fighting racism and sexism to a group whose sole purpose for existing is to keep down black people? \n\nBLM is racists towards white people, Femenism is Sexist towards men.  \n   \nat least the KKK is honest.'
 "It's great to see blacks trying to be like whites so much lately. I thought they all hated us."
 'Black family with a  father!'
 "That's because all black people look alike."
 "When white people leave an area....white flight, racist.\n\nWhen white people move in to an area....gentrification, racist.\n\nWhe